In [22]:
import os
import pandas as pd
from langchain_openai import OpenAI
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
from langchain_community.agent_toolkits import create_sql_agent
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate

# ###### API key #############
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [23]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

In [47]:
from langchain.memory.buffer import ConversationBufferMemory
from langchain.chains.conversation.base import ConversationChain

# Initialize ConversationBufferMemory
memory = ConversationBufferMemory(
    session_id="your_session_id",  # Identifies your user or a user's session
    memory_key="history",          # Ensure this matches the key used in chain's prompt template
    return_messages=True,          # Does your prompt template expect a string or a list of Messages?
)
# # Wrap the SQL Query Chain with a ConversationChain
# # , chain=chain_sql
prompt3 = PromptTemplate(
    template="""The following is a friendly conversation between a human and an AI. The AI is smart, technical, understands and generates sql queries, talkative, provides lots of specific details from its context and replies to Human in SQL language. When human asks a question to AI or send a query to the AI, the AI refers to a table named 'asian_paints' to generate the sql query for the reply. The AI is also given 7 columns, column's datatype and example values of the columns of the table asian_paints seperated by comma as following:
    1. Region, object, 'Region 1'
    2. Pincode, int64, '123456'
    3. Date, datetime64[ns], '2024-06-04 00:00:00.000000'
    4. Skues, object, 'SKU1'
    5. Demand, int64, 250000
    6. Inventory, int64, 50000
    7. Sales, int64, 270000
    The AI should also check if the syntax of SQL query The AI generated is correct or not to run it on sqlite db for results. In case of date filters present in query, the format of dates have only date values not time e.g. 'YYYY-MM-DD'. Change the format of these date values to default python tiemstamp datetime64[ns], e.g. '2024-06-04 00:00:00.000000'.
    If the AI does not know the answer to a question, it truthfully says it does not know. Use the following format:
    response: SQL Query generated by the AI as reply to Human

Current conversation:
{history}
Human: {input}
AI Assistant:
""",
    input_variables=["input", "history"]
)
conversation_chain = ConversationChain(llm=llm,prompt=prompt3, memory=memory,verbose=True)

    # Also, the content inside the reply of AI should neither include the prefix '```sql\n' nor the suffix '\n```'.

In [48]:
# output2 = conversation_chain.invoke({"input" : "Get Sales for Skue8 for Region 1"})
output = conversation_chain.predict(input="Get Sales for Skue8 for Region 1")
o2 = output.split('```sql\n')[-1]
final_output = o2.split('\n```')[0]
final_output



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is smart, technical, understands and generates sql queries, talkative, provides lots of specific details from its context and replies to Human in SQL language. When human asks a question to AI or send a query to the AI, the AI refers to a table named 'asian_paints' to generate the sql query for the reply. The AI is also given 7 columns, column's datatype and example values of the columns of the table asian_paints seperated by comma as following:
    1. Region, object, 'Region 1'
    2. Pincode, int64, '123456'
    3. Date, datetime64[ns], '2024-06-04 00:00:00.000000'
    4. Skues, object, 'SKU1'
    5. Demand, int64, 250000
    6. Inventory, int64, 50000
    7. Sales, int64, 270000
    The AI should also check if the syntax of SQL query The AI generated is correct or not to run it on sqlite db for results. In case of date filters present in que

"SELECT Sales\nFROM asian_paints\nWHERE Skues = 'SKU8' AND Region = 'Region 1';"

In [49]:
conversation_chain.predict(input="Get Sales for Skue8 for Region 2")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is smart, technical, understands and generates sql queries, talkative, provides lots of specific details from its context and replies to Human in SQL language. When human asks a question to AI or send a query to the AI, the AI refers to a table named 'asian_paints' to generate the sql query for the reply. The AI is also given 7 columns, column's datatype and example values of the columns of the table asian_paints seperated by comma as following:
    1. Region, object, 'Region 1'
    2. Pincode, int64, '123456'
    3. Date, datetime64[ns], '2024-06-04 00:00:00.000000'
    4. Skues, object, 'SKU1'
    5. Demand, int64, 250000
    6. Inventory, int64, 50000
    7. Sales, int64, 270000
    The AI should also check if the syntax of SQL query The AI generated is correct or not to run it on sqlite db for results. In case of date filters present in que

"```sql\nSELECT Sales\nFROM asian_paints\nWHERE Skues = 'SKU8' AND Region = 'Region 2';\n```"

In [50]:
conversation_chain.predict(input="For Skue 10, get the difference of sales between 4th june and 13th june")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is smart, technical, understands and generates sql queries, talkative, provides lots of specific details from its context and replies to Human in SQL language. When human asks a question to AI or send a query to the AI, the AI refers to a table named 'asian_paints' to generate the sql query for the reply. The AI is also given 7 columns, column's datatype and example values of the columns of the table asian_paints seperated by comma as following:
    1. Region, object, 'Region 1'
    2. Pincode, int64, '123456'
    3. Date, datetime64[ns], '2024-06-04 00:00:00.000000'
    4. Skues, object, 'SKU1'
    5. Demand, int64, 250000
    6. Inventory, int64, 50000
    7. Sales, int64, 270000
    The AI should also check if the syntax of SQL query The AI generated is correct or not to run it on sqlite db for results. In case of date filters present in que

"```sql\nSELECT \n    (SELECT Sales \n     FROM asian_paints \n     WHERE Skues = 'SKU10' AND Date = '2024-06-13 00:00:00.000000') - \n    (SELECT Sales \n     FROM asian_paints \n     WHERE Skues = 'SKU10' AND Date = '2024-06-04 00:00:00.000000') AS Sales_Difference;\n```"

In [17]:
print('outie2:',output2['response'])
response = output2['response']
response_split = response.split('```sql\n')[-1]

split2 = response_split.split('\n```')
# final_query = split2[0]
print(split2)

outie2: ```sql
SELECT Sales
FROM asian_paints
WHERE Skues = 'SKU8' AND Region = 'Region 1';
```
["SELECT Sales\nFROM asian_paints\nWHERE Skues = 'SKU8' AND Region = 'Region 1';", '']
